### 爬取浙江公务员职位信息

In [2]:
import urllib
import pandas as pd

from bs4 import BeautifulSoup

### 1 爬取页面

In [3]:
api = 'http://zw.huatu.com'
base='/2018/'
url=api+base

header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3236.0 Safari/537.36'}
request = urllib.request.Request(url,headers=header)
response = urllib.request.urlopen(request).read()

content=BeautifulSoup(response, 'lxml')

### 2 内容解析
- 省/市 : 对应 URL [32个]

In [5]:
areas_txt=content.find_all('p', attrs={'id':'ydiqu'})[0]
area_metas_lis=[]
counter=0

for i in areas_txt.find_all('a', attrs={'target':'_blank'}):
    url=api+i.get_attribute_list('href')[0][2:]
    area=i.get_text()
    counter+=1
    area_metas_lis.append([counter,area,url])

In [12]:
df=pd.DataFrame(area_metas_lis, columns=['no','area', 'url'])
df.head()

,no,area,url
0,1,安徽,http://zw.huatu.com/2018/buweisearch/1.html
1,2,北京,http://zw.huatu.com/2018/buweisearch/2.html
2,3,福建,http://zw.huatu.com/2018/buweisearch/3.html
3,4,甘肃,http://zw.huatu.com/2018/buweisearch/4.html
4,5,广东,http://zw.huatu.com/2018/buweisearch/5.html


### 3 以浙江为例，通过省份查询招聘单位及具体信息访问途径（URL）

In [8]:
query_area='浙江'
df[df.area==query_area]

,no,area,url
30,31,浙江,http://zw.huatu.com/2018/buweisearch/31.html


In [9]:
url2=df[df.area==query_area].url.values[0]

request2 = urllib.request.Request(url2,headers=header)
response2 = urllib.request.urlopen(request2).read()
content2=BeautifulSoup(response2, 'lxml')

tmp2=content2.find_all('table', attrs={'cellspacing':'0','width':'100%'})[0]

unit_lis=[] # 存储职位列表
for i in tmp2.find_all('a'):
    t_url=api+'/2018'+i.get_attribute_list('href')[0][2:]
    unit_lis.append([i.get_text().strip(),t_url]) # strip 去除空格

unit_df=pd.DataFrame(unit_lis,columns=['unit_name','url'])

In [13]:
unit_df.head()

,unit_name,url
0,国家物资储备局浙江办事处,http://zw.huatu.com/2018/buwei2018/1763.html
1,浙江海事局,http://zw.huatu.com/2018/buwei2018/2009.html
2,杭州海关,http://zw.huatu.com/2018/buwei2018/2105.html
3,宁波海关,http://zw.huatu.com/2018/buwei2018/2107.html
4,浙江省国家税务局,http://zw.huatu.com/2018/buwei2018/2175.html


### 4 查询所有部门的招聘岗位等详细信息

In [10]:
'''
获取岗位详细信息
url :
    访问路径
'''
def get_post_metas(url):
    tmp_request = urllib.request.Request(url,headers=header)
    tmp_response = urllib.request.urlopen(tmp_request).read()
    tmp_content=BeautifulSoup(tmp_response, 'lxml')
    
    td_lis=[i.get_text() for i in tmp_content.find_all('td')]
    internal=[]
    for i in range(len(td_lis)):
        if i%10==0: # 生成间隔区间
            internal.append([i,i+10])
    
    row_lis=[]
    for lt,rt in internal: # 根据间隔区间，将数据分行
        row_lis.append(td_lis[lt:rt])
        
    return row_lis

post_metas_lis=[]

for unit_name,url in unit_lis: # 循环，加工招聘单位所招岗位详细信息
    post_metas_lis+=get_post_metas(url)

# 将 list 转换为 DataFrame 格式

# '部门名称','用人用司','职位名称','要求专业','招考人数','报考人数','历年分数线','历年竞争比','录取概率','对比'
post_metas_df=pd.DataFrame(post_metas_lis, columns=['unit_name','employee_unit','post_name','professional','person_num','person_num2','view','view2','detail','compare'])

In [22]:
post_metas_df[post_metas_df.unit_name=='浙江省国家税务局'].head(3)

,unit_name,employee_unit,post_name,professional,person_num,person_num2,view,view2,detail,compare
100,浙江省国家税务局,杭州市上城区国家税务局,纳税服务科科员（一）,公共管理类,1,0,查看,查看,详情,对比
101,浙江省国家税务局,杭州市上城区国家税务局,纳税服务科科员（二）,工商管理类,1,0,查看,查看,详情,对比
102,浙江省国家税务局,杭州市上城区国家税务局,纳税服务科科员（三）,财政学类,1,0,查看,查看,详情,对比


In [249]:
# 保存数据信息

post_metas_df.to_csv('./post_metas.csv')